# **Deep LLMs - Proceso de Extracción y Limpieza de Ubicaciones en Resúmenes de Tiroteos**

Al analizar los datos del dataset de tiroteos, me di cuenta de que había tiroteos en los que aparecía vacío el campo de Location, a pesar de mencionarse la ubicación del tiroteo en el campo Summary.

El objetivo principal de este apartado es identificar y extraer automáticamente las ubicaciones mencionadas en los resúmenes de incidentes de tiroteos, y posteriormente limpiar y corregir cualquier anomalía en los datos extraídos.

Una vez obtenidas estas ubiaciones, se añadirán en una nueva columna del dataframe para su uso en posteriores análisis.

También, podremos comprobar si la ubicación extraída del resumen coincide con la localización de aquellos tiroteos que no tienen el campo Location vacío.

Para llevar a cabo esta tarea, hemos utilizado una combinación de herramientas de procesamiento de lenguaje natural (NLP) y técnicas de manipulación de datos.

A continuación, mencionamos las principales herramientas y métodos utilizados:

- Python y Pandas:

  - Utilizamos Python como el lenguaje principal para el procesamiento de datos.

  - La biblioteca Pandas se empleó para la carga, manipulación y almacenamiento de datos en formato CSV.

- Transformers de Hugging Face:

  - Empleamos el modelo *dbmdz/bert-large-cased-finetuned-conll03-english*, un modelo de transformers preentrenado para el reconocimiento de entidades nombradas (NER), capaz de identificar entidades como ubicaciones, nombres de personas y organizaciones en el texto.

  - La función pipeline de Hugging Face se utilizó para crear un pipeline de NER, que facilita la identificación y extracción de ubicaciones.

- Tokenización y Limpieza de Datos:

  - Los modelos de transformers a menudo descomponen palabras en subpalabras, marcando las continuaciones con un prefijo ##. Implementamos una lógica específica para unir estas subpalabras y formar palabras completas.

  - Creamos una función personalizada para procesar los tokens, eliminando caracteres no deseados y uniendo subpalabras adecuadamente.

- Procesamiento por Lotes:
  - Para manejar de manera eficiente el procesamiento de grandes volúmenes de texto, dividimos el dataset en lotes y procesamos cada lote de manera iterativa.

**PRUEBA 1**

Este código realiza un análisis de texto utilizando el modelo de reconocimiento de entidades nombradas (NER) de Hugging Face para extraer ubicaciones de los resúmenes de incidentes de tiroteos.
El script carga el dataset, utiliza el modelo de transformers preentrenado para identificar y extraer las ubicaciones mencionadas en los resúmenes, y guarda los resultados en un nuevo archivo CSV.

**Problema:**
Durante el proceso de extracción de ubicaciones utilizando modelos de transformers como BERT, es común encontrar caracteres ## delante de algunas palabras. Esto se debe al proceso de tokenización de subpalabras utilizado por estos modelos.
Los tokens que comienzan con ## representan subpalabras que se deben unir a la palabra anterior. Por ejemplo, "Mandalay Bay" es tokenizado como "Man ##ada ##lay Bay", por lo que debemos solucionar este problema para poder emplear bien los datos.

In [1]:
# Importamos las bibliotecas necesarias
import pandas as pd
from transformers import pipeline
import torch

# Cargamos el dataset
file_path = 'Mass Shootings Dataset Ver 5.csv'  # Ruta del archivo CSV subido a Google Colab
df = pd.read_csv(file_path, encoding='latin1')  # Cargamos el CSV con codificación 'latin1'

# Creamos el pipeline de NER usando un modelo preentrenado y GPU si está disponible
device = 0 if torch.cuda.is_available() else -1    # Usamos GPU si está disponible, de lo contrario usar CPU
ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english", device=device)

# Definimos una función para extraer ubicaciones de los summaries
def extract_locations_batch(texts):
    all_locations = []  # Lista para almacenar todas las ubicaciones extraídas
    for text in texts:
        ner_results = ner_pipeline(text)  # Aplicamos el modelo NER al texto
        # Filtramos las entidades identificadas como ubicaciones (LOC) y la unimos en una cadena
        locations = [result['word'] for result in ner_results if 'LOC' in result['entity']]
        all_locations.append(' '.join(locations))  # Añadimos las ubicaciones a la lista
    return all_locations  # Devolvemos la lista de ubicaciones extraídas

# Dividimos el dataset en lotes y acumulamos los resultados
batch_size = 10  # Tamaño del lote
all_extracted_locations = []  # Lista para acumular todas las ubicaciones extraídas

# Procesamos el dataset en lotes
for start in range(0, len(df), batch_size):
    end = min(start + batch_size, len(df))  # Definimos el rango del lote actual
    batch_texts = df['Summary'][start:end].tolist()  # Extraemos el texto del lote actual
    # Extraemos las ubicaciones del lote actual y acumulamos los resultados
    all_extracted_locations.extend(extract_locations_batch(batch_texts))

# Añadimos la columna de ubicaciones extraídas al dataframe
df['Extracted_Location'] = all_extracted_locations

# Guardamos el dataframe resultante en un archivo CSV
output_file_path = 'Extracted_Locations.csv'
df.to_csv(output_file_path, index=False)  # Guardamos el dataframe en un archivo CSV sin el índice

# Mostramos las primeras filas del dataframe con la nueva columna
print(df[['Summary', 'Extracted_Location']].head())

# Descargamos el archivo CSV
from google.colab import files  # Importamos la biblioteca para descargar archivos en Google Colab
files.download('Extracted_Locations.csv')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you ar

                                             Summary  \
0  Devin Patrick Kelley, 26, an ex-air force offi...   
1  Scott Allen Ostrem, 47, walked into a Walmart ...   
2  Radee Labeeb Prince, 37, fatally shot three pe...   
3  Stephen Craig Paddock, opened fire from the 32...   
4  Jimmy Lam, 38, fatally shot three coworkers an...   

              Extracted_Location  
0                          Texas  
1                  Denver Denver  
2  Baltimore Wilmington Delaware  
3            Man ##ada ##lay Bay  
4                  San Francisco  


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**PRUEBA 2**

Este código realiza un análisis de texto utilizando el modelo de reconocimiento de entidades nombradas (NER) de Hugging Face para extraer y limpiar ubicaciones de los resúmenes de incidentes de tiroteos.
El script carga el dataset, utiliza un modelo de transformers preentrenado para identificar y extraer las ubicaciones mencionadas en los resúmenes, limpia los tokens de subpalabras, y guarda los resultados en un nuevo archivo CSV.

**Problema:**
Hemos eliminado los caracteres ## delante de las palabras, pero ahora estas subpalabras no se unen adecuadamente y resultan en palabras separadas por espacios. Por ejemplo, "Man ##ada ##lay Bay" ha pasado a ser "Man ada lay Bay", pero todavía no es "Mandalay Bay", que es lo que buscamos para el correcto análisis en R de las localizaciones.

In [2]:
# Importamos las bibliotecas necesarias
import pandas as pd
from transformers import pipeline
import torch

# Cargamos el dataset
file_path = 'Mass Shootings Dataset Ver 5.csv'  # Ruta del archivo CSV en Google Colab
df = pd.read_csv(file_path, encoding='latin1')  # Cargamos el CSV con codificación 'latin1'

# Creamos el pipeline de NER usando un modelo preentrenado y GPU si está disponible
device = 0 if torch.cuda.is_available() else -1  # Usamos GPU si está disponible, de lo contrario usar CPU
ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english", device=device)

# Definimos una función para extraer y limpiar ubicaciones de los summaries
def extract_and_clean_locations(text):
    ner_results = ner_pipeline(text)  # Aplicamos el modelo NER al texto
    locations = []
    for result in ner_results:
        if 'LOC' in result['entity']:
            # Limpiamos los tokens de ubicaciones, eliminando los caracteres '#' de subpalabras
            word = result['word'].replace('#', '')
            locations.append(word)

    # Unimos subpalabras
    cleaned_locations = ' '.join(locations).replace(' ##', '').replace('##', '')
    return cleaned_locations  # Devolvemos las ubicaciones limpias

# Dividimos el dataset en lotes y acumulamos los resultados
batch_size = 10  # Tamaño del lote
all_extracted_locations = []  # Lista para acumular todas las ubicaciones extraídas y limpiadas

# Procesamos el dataset en lotes
for start in range(0, len(df), batch_size):
    end = min(start + batch_size, len(df))  # Definimos el rango del lote actual
    batch_texts = df['Summary'][start:end].tolist()  # Extraemos el texto del lote actual
    # Extraemos y limpiamos las ubicaciones del lote actual y acumulamos los resultados
    all_extracted_locations.extend([extract_and_clean_locations(text) for text in batch_texts])

# Añadimos la columna de ubicaciones extraídas y limpiadas al dataframe
df['Extracted_Location'] = all_extracted_locations

# Guardamos el dataframe resultante en un archivo CSV
output_file_path = 'Extracted_Locations_Clean.csv'
df.to_csv(output_file_path, index=False)  # Guardamos el dataframe en un archivo CSV sin el índice

# Mostramos las primeras filas del dataframe con la nueva columna
print(df[['Summary', 'Extracted_Location']].head())

# Descargamos el archivo CSV
from google.colab import files  # Importamos la biblioteca para descargar archivos en Google Colab
files.download('Extracted_Locations_Clean.csv')

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


                                             Summary  \
0  Devin Patrick Kelley, 26, an ex-air force offi...   
1  Scott Allen Ostrem, 47, walked into a Walmart ...   
2  Radee Labeeb Prince, 37, fatally shot three pe...   
3  Stephen Craig Paddock, opened fire from the 32...   
4  Jimmy Lam, 38, fatally shot three coworkers an...   

              Extracted_Location  
0                          Texas  
1                  Denver Denver  
2  Baltimore Wilmington Delaware  
3                Man ada lay Bay  
4                  San Francisco  


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**PRUEBA 3**

Este código utiliza el modelo de reconocimiento de entidades nombradas (NER) de Hugging Face para extraer y limpiar ubicaciones de los resúmenes de incidentes de tiroteos.
El script carga el dataset, utiliza un modelo de transformers preentrenado para identificar y extraer las ubicaciones mencionadas en los resúmenes, limpia los tokens de subpalabras, une las palabras correctamente y guarda los resultados en un nuevo archivo CSV.

Ya hemos solucionado todos los problemas surgidos y hemos obtenido, en aquellos tiroteos que mencionan la ubicación en el resumen, el lugar donde ocurrió el incidente.

In [3]:
# Importamos las bibliotecas necesarias
import pandas as pd
from transformers import pipeline
import torch

# Cargamos el dataset
file_path = 'Mass Shootings Dataset Ver 5.csv'  # Ruta del archivo CSV en Google Colab
df = pd.read_csv(file_path, encoding='latin1')  # Cargamos el CSV con codificación 'latin1'

# Creamos el pipeline de NER usando un modelo preentrenado y GPU si está disponible
device = 0 if torch.cuda.is_available() else -1  # Usamos GPU si está disponible, de lo contrario usar CPU
ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english", device=device)

# Definimos una función para extraer y limpiar ubicaciones de los summaries
def extract_and_clean_locations(text):
    ner_results = ner_pipeline(text)  # Aplicamos el modelo NER al texto
    locations = []  # Lista para almacenar las ubicaciones extraídas
    for result in ner_results:
        if 'LOC' in result['entity']:
            word = result['word']
            if word.startswith('##'):
                # Si el token empieza con ##, se une al token anterior
                if locations:
                    locations[-1] += word[2:]  # Unimos subpalabra a la palabra anterior
                else:
                    locations.append(word[2:])  # Si es el primer token, solo eliminamos ##
            else:
                locations.append(word)

    cleaned_locations = ' '.join(locations)  # Unimos las palabras con espacios
    return cleaned_locations  # Devolvemos las ubicaciones limpias

# Dividimos el dataset en lotes y acumulamos los resultados
batch_size = 10  # Tamaño del lote
all_extracted_locations = []  # Lista para acumular todas las ubicaciones extraídas y limpiadas

# Procesamos el dataset en lotes
for start in range(0, len(df), batch_size):
    end = min(start + batch_size, len(df))  # Definimos el rango del lote actual
    batch_texts = df['Summary'][start:end].tolist()  # Extraemos el texto del lote actual
    # Extraemos y limpiamos las ubicaciones del lote actual y acumulamos los resultados
    all_extracted_locations.extend([extract_and_clean_locations(text) for text in batch_texts])

# Añadimos la columna de ubicaciones extraídas y limpiadas al dataframe
df['Extracted_Location'] = all_extracted_locations

# Guardamos el dataframe resultante en un archivo CSV
output_file_path = 'Extracted_Locations_Clean_2.csv'
df.to_csv(output_file_path, index=False)  # Guardamos el dataframe en un archivo CSV sin el índice

# Mostramos las primeras filas del dataframe con la nueva columna
print(df[['Summary', 'Extracted_Location']].head())

# Descargamos el archivo CSV
from google.colab import files  # Importamos la biblioteca para descargar archivos en Google Colab
files.download('Extracted_Locations_Clean_2.csv')

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


                                             Summary  \
0  Devin Patrick Kelley, 26, an ex-air force offi...   
1  Scott Allen Ostrem, 47, walked into a Walmart ...   
2  Radee Labeeb Prince, 37, fatally shot three pe...   
3  Stephen Craig Paddock, opened fire from the 32...   
4  Jimmy Lam, 38, fatally shot three coworkers an...   

              Extracted_Location  
0                          Texas  
1                  Denver Denver  
2  Baltimore Wilmington Delaware  
3                  Manadalay Bay  
4                  San Francisco  


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>